# Statoil

In [ ]:
from google.colab import files
files.upload()

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c statoil-iceberg-classifier-challenge

In [ ]:
!7z e sample_submission.csv.7z
!7z e test.json.7z
!7z e train.json.7z

In [7]:
!ls

kaggle.json  sample_submission.csv     test.json     train.json
sample_data  sample_submission.csv.7z  test.json.7z  train.json.7z


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab

plt.rcParams['figure.figsize'] = 10, 10

In [9]:
train = pd.read_json('train.json')
target_train = train['is_iceberg']
test = pd.read_json('test.json')

In [13]:
train.shape, test.shape

((1604, 5), (8424, 4))

In [10]:
target_train = train['is_iceberg']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce') # erros='coerce' 에러가 발생하면 NaN으로 변환
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
train['inc_angle'] = train['inc_angle'].fillna(method='pad')
# method='pad' 결측치 앞의 값으로 채워넣기

X_angle = train['inc_angle']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle = test['inc_angle']

In [11]:
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])
X_band_3 = (X_band_1 + X_band_2)

In [24]:
X_band_1.shape, X_band_2.shape, X_band_3.shape

((1604, 75, 75), (1604, 75, 75), (1604, 75, 75))

In [12]:
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis],
                          X_band_2[:, :, :, np.newaxis],
                          X_band_3[:, :, :, np.newaxis]], axis=-1)

In [36]:
X_train.shape

(1604, 75, 75, 3)

In [13]:
X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3 = (X_band_test_1 + X_band_test_2) / 2

In [14]:
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis],
                         X_band_test_2[:, :, :, np.newaxis],
                         X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [15]:
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [41]:
# Data Augmentation.
batch_size=64
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)
# 기존 이미지에 변형을 가해 새로운 데이터 셋을 만드는 함수

In [59]:
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    # flow에 X와 y 값을 넣어 (X, y) 또는 sample_weight가 None이 아닐경우 (X, y, sample_weight) 반환

    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[1]], X1i[1]
        # [genX1의 X, genX2의 y], genX1의 y 

In [60]:
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

In [64]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [62]:
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    # 샘플을 뽑을 때 label들의 비율을 맞추기 위함

    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
            gen_flow,
            steps_per_epoch=24,
            epochs=100,
            shuffle=True,
            verbose=1,
            validation_data=([X_holdout,X_angle_hold], Y_holdout),
            callbacks=callbacks)

        galaxyModel.load_weights(filepath=file_path)

        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])

        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [65]:
getVggAngleModel().summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 75, 75, 64)   1792        input_17[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 75, 75, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 37, 37, 64)   0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [66]:
preds = myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
24/24 [==============================] - 8s 291ms/step - loss: 1.2093 - accuracy: 0.5150 - val_loss: 0.5488 - val_accuracy: 0.6916
Epoch 2/100
24/24 [==============================] - 5s 213ms/step - loss: 0.6694 - accuracy: 0.6195 - val_loss: 0.4494 - val_accuracy: 0.7720
Epoch 3/100
24/24 [==============================] - 5s 212ms/step - loss: 0.5589 - accuracy: 0.6979 - val_loss: 0.4195 - val_accuracy: 0.8542
Epoch 4/100
24/24 [==============================] - 5s 213ms/step - loss: 0.4212 - accuracy: 0.7996 - val_loss: 0.3879 - val_accuracy: 0.8037
Epoch 5/100
24/24 [==============================] - 5s 210ms/step - loss: 0.3619 - accuracy: 0.8304 - val_loss: 0.3535 - val_accuracy: 0.8542
Epoch 6/100
24/24 [==============================] - 5s 215ms/step - loss: 0.3891 - accuracy: 0.8103 - val_loss: 0.3487 - val_accuracy: 0.8430
Epoch 7/100
24/24 [==============================] - 5s 218ms/step - loss: 0.3363 - accuracy: 0.8508 - val_loss: 0.3048 - val_accuracy: 0.8636

## reference

- [Transfer Learning with VGG-16 CNN+AUG LB 0.1712](https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712)